<a href="https://colab.research.google.com/github/Mayank-Bansal24/LAPLSTM-Models/blob/main/manual_balancet01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses as lo_ss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import string
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.utils import class_weight
from sklearn.metrics import classification_report , roc_auc_score
import keras 
import keras.backend as K
from google.colab import drive
from numpy import savetxt
import os

In [57]:
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [58]:
t=1

In [59]:
os.chdir('/content/gdrive/MyDrive/laplstm/manual_balance/t'+str(t))

In [60]:
import os
github_raw_link="https://raw.githubusercontent.com/Aryansh085/LAP/main/858417d1-7d54-4115-a01b-fdda5e03ada3_testing_combined_rows4035_disc_1_0p9_MULTIPLY_preproces_155cols.csv"
df1=pd.read_csv((github_raw_link),header = None)
dftemp=np.array(df1)
dftemp.shape
dftemp2=dftemp[:,5:]
dftemp2=np.array(dftemp2)
print(dftemp2.shape)

(4035, 150)


In [61]:
# dftemp2
dftemp2.shape
# dftemp2=pd.DataFrame(dftemp2)
# dftemp2.head()

(4035, 150)

--> muliply the reward of that instance into the loss of that instance to give weight

In [62]:
from sklearn.preprocessing import StandardScaler

slr=StandardScaler()
slr.fit(dftemp2)
dftemp2 = slr.transform(dftemp2)

In [63]:
# f1-score function
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
    

In [64]:
y=np.array(dftemp[:,0])
print(y)
y_new=y[t:]
y_new.shape


[2. 0. 0. ... 0. 2. 2.]


(4034,)

In [65]:
x=np.array(dftemp2)
# print(x)
# print(x.shape)
x.shape,y_new.shape

((4035, 150), (4034,))

In [66]:
x_3d=[]
for i in range(4035-t):
  tmp=[]
  for j in range(t):
    tmp.append(x[j+i])
  x_3d.append(tmp)
x_3d= np.array(x_3d)
x_3d.shape

(4034, 1, 150)

In [67]:
train_test_ratio=0.8
print(x_3d.shape[0])
x_train=[]
y_train=[]
x_test=[]
y_test=[]
for i in range(x_3d.shape[0]):
  l=[]
  ly=[]
  no=np.random.randint(100)
  if(no<80):
    l=x_3d[i]
    ly=y_new[i]
    x_train.append(l)
    y_train.append(ly)
  else:
    l=x_3d[i]
    ly=y_new[i]
    x_test.append(l)
    y_test.append(ly)


x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)
x_train.shape,y_train.shape,x_test.shape,y_test.shape

4034


((3219, 1, 150), (3219,), (815, 1, 150), (815,))

In [68]:
cnt0=0
cnt1=0
cnt2=0
for i in range(x_train.shape[0]):
  if(y_train[i]==0):
    cnt0+=1
  elif(y_train[i]==1):
    cnt1+=1
  elif(y_train[i]==2):
    cnt2+=1
print(cnt0)
print(cnt1)
print(cnt2)  

1448
853
918


In [69]:
x_train_1=[]
y_train_1=[]
c1=0
c2=0
c3=0
for i in range(x_train.shape[0]):
  if(y_train[i]==0):
    x_train_1.append(x_train[i])
    y_train_1.append(0)
    c1+=1
  elif(y_train[i]==1):
    x_train_1.append(x_train[i])
    y_train_1.append(1)
    c2+=1
  elif(y_train[i]==2 ):
    x_train_1.append(x_train[i])
    y_train_1.append(2)
    c3=c3+1
for i in range(x_train.shape[0]):
  if(y_train[i]==1):
    no=np.random.randint(100)
    if(no>=30):
      x_train_1.append(x_train[i])
      y_train_1.append(1)
      c2+=1
  elif(y_train[i]==2 ):
    no=np.random.randint(100)
    if(no>=27):
      x_train_1.append(x_train[i])
      y_train_1.append(2)
      c3+=1

y_train=[]
y_train=y_train_1
x_train=[]
x_train=x_train_1
x_train=np.array(x_train)
y_train=np.array(y_train)
print(c1)
print(c2)
print(c3)

1448
1448
1599


In [70]:

# model = Sequential()
# model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=( t,150,)))
# model.add(LSTM(100, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(25, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(10, activation='relu'))
# model.add(Dense(3))
# model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=1)
# test_output = model.predict(x_test, verbose=1)
# # history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)
# # test_output = model.predict(x=, verbose=0)
# # print(test_output)

In [71]:
# sklearn_weights={0:1738,1:2348,2:2362}
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=( t,150,)))
model.add(LSTM(400, activation='relu', return_sequences=True))
model.add(LSTM(800, activation='relu', return_sequences=True))
model.add(LSTM(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3,activation='sigmoid'))
optimi=keras.optimizers.Adam()
model.compile(optimizer=optimi, loss='sparse_categorical_crossentropy',metrics=['accuracy',get_f1])

history = model.fit(x_train, y_train, epochs=25, validation_split=0.2, verbose=1)
# history = model.fit(x_train, y_train, epochs=25, validation_split=0.2, verbose=1,class_weight=sklearn_weights)
test_output = model.predict(x_test, verbose=1)


Epoch 1/25
113/113 [==============================] - 6s 22ms/step - loss: 1.0940 - accuracy: 0.4002 - get_f1: 1.1078 - val_loss: 1.1700 - val_accuracy: 0.0000e+00 - val_get_f1: 1.5515
Epoch 2/25
113/113 [==============================] - 2s 17ms/step - loss: 1.0827 - accuracy: 0.4027 - get_f1: 1.1243 - val_loss: 1.2092 - val_accuracy: 0.0000e+00 - val_get_f1: 1.7426
Epoch 3/25
113/113 [==============================] - 2s 17ms/step - loss: 1.0373 - accuracy: 0.4399 - get_f1: 1.1631 - val_loss: 1.1054 - val_accuracy: 0.2492 - val_get_f1: 1.7077
Epoch 4/25
113/113 [==============================] - 2s 17ms/step - loss: 0.8933 - accuracy: 0.5412 - get_f1: 1.1086 - val_loss: 0.9769 - val_accuracy: 0.4483 - val_get_f1: 1.6735
Epoch 5/25
113/113 [==============================] - 2s 17ms/step - loss: 0.7269 - accuracy: 0.6285 - get_f1: 1.0543 - val_loss: 0.8182 - val_accuracy: 0.5028 - val_get_f1: 1.5760
Epoch 6/25
113/113 [==============================] - 2s 17ms/step - loss: 0.5749 - acc

In [72]:
test_output_1=np.array([np.argmax(i)  for i in test_output])
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,test_output_1))
test_output=pd.DataFrame(test_output)
print(accuracy_score(y_test,test_output_1))


[[167 102 110]
 [ 80  46  76]
 [ 98  52  84]]
0.36441717791411044


In [73]:
print(get_f1(y_test.astype('float32'), test_output_1.astype('float32')))

tf.Tensor(0.5695364, shape=(), dtype=float32)


In [74]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
savetxt('arrxtrain.csv', x_train_reshaped, delimiter=',')
savetxt('arrytrain.csv', y_train, delimiter=',')
savetxt('arrxtest.csv', x_test_reshaped, delimiter=',')
savetxt('arrytest.csv', y_test, delimiter=',')